<a href="https://colab.research.google.com/github/movie112/INU-DILAB/blob/main/lab_11_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab_11_4: Time Series
## contents
- time series data
- apply RNN
  - Many-to-one
  - data reading
  - neural net setting
  - training & evaluation
- exercise
---
## time series data
- 일정한 시간 간격으로 배치된 data

## apply RNN
### 1. Many-to-one
example: 7일간 data로 다음 날 종가 예측하는 모델
- 전제: 8일차의 종가를 예측하기 위해서 그 전 일주일을 보면 될 것이다.

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FchmN6B%2Fbtq0UcFqOkk%2F1SERgahinWKGlblZw2DyH0%2Fimg.png" width="500px" height="300px"></img>

> 1. dim=5인 vector가 첫 날input으로 들어가서 cell에서 처리된 정보 다음 cell로 전달
> 2. 2일차는 input + 전날 처리된 정보를 입력받아 cell에서 처리하고 다음 cell로 전달   
> ...
> 3. 7번 data 유통하고 마지막에 output으로 8일 종가 예측
</br>

- 8일차 종가: dim=1 짜리 vector
- 바로 8일차 종가 처리한다면 매 단계 hidden state도 dim=1
  - model 부담 -> 분산 1. 데이터 유통단계(hidden state) 2. label 맞추는 단계
  - 출력 단계에서는 FC를 연결해서 이 output이 종가 맞추도록 구성

### 2. data reading


In [ ]:
import torch
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

# Random seed to make results deterministic and reproducible
torch.manual_seed(0)

# hyper parameters
seq_length = 7    # 일 수
data_dim = 5      # 5개의 dim: 시가, 최고가, 최저가, 거래량, 종가
hidden_dim = 10
output_dim = 1    # 종가의 dim
learning_rate = 0.01
iterations = 500


# load data
xy = np.loadtxt("data-02-stock_daily.csv", delimiter=",")
xy = xy[::-1]  # reverse order, 시간 역순

# 70% 는 train set, 나머지는 test set
train_size = int(len(xy) * 0.7)
train_set = xy[0:train_size]
test_set = xy[train_size - seq_length:]

# scaling data
# 상대값으로 바꾸어 [0,1]로 나타낸다. (차이가 큰 경우 학습의 부담을 줄여주기 위해서)
train_set = minmax_scaler(train_set)
test_set = minmax_scaler(test_set)

# make train-test dataset to input
trainX, trainY = build_dataset(train_set, seq_length)
testX, testY = build_dataset(test_set, seq_length)


trainX_tensor = torch.FloatTensor(trainX)
trainY_tensor = torch.FloatTensor(trainY)

testX_tensor = torch.FloatTensor(testX)
testY_tensor = torch.FloatTensor(testY)

### 3. neural net setting

In [ ]:
# neural network 선언 
class Net(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, layers):
        super(Net, self).__init__()
        self.rnn = torch.nn.LSTM(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, output_dim, bias=True)

    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x[:, -1])
        return x


net = Net(data_dim, hidden_dim, output_dim, 1)

# loss & optimizer setting
criterion = torch.nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

### 4. training & evaluation

In [ ]:
# start training
for i in range(iterations):

    optimizer.zero_grad()
    outputs = net(trainX_tensor)
    loss = criterion(outputs, trainY_tensor)
    loss.backward()
    optimizer.step()
    print(i, loss.item())
    
  
plt.plot(testY)
plt.plot(net(testX_tensor).data.numpy())
plt.legend(['original', 'prediction'])
plt.show()

## exercise
- implement stock prediction right now?
- use more features to improve robustness